In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("./../../")

In [ ]:
from pathlib import Path
import math
import pickle
import pprint
import time
import datetime
#
import torch
import torchvision
from torchvision import utils
import torchvision.transforms as T
import torchvision.datasets as datasets
#
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tqdm import tqdm
from dotted_dict import DottedDict
#
from misc.plot_utils import plot_mat, imshow
from effcn.functions import max_norm_masking
from effcn.models_mnist import Backbone, Decoder, EffCapsNet, CNN_CR_SF, CNN_CR, CNN_R
from misc.utils import mkdir_directories

In [ ]:
a = torch.tensor([[4.,2.,6.],[1., 3.,2.],[2., 3.,2.]])
print(a.shape)
dim = 1
print(torch.var(a, dim=dim))
print(torch.cov(a))


#a = np.array([[1, 2], [3, 4]])
print(np.var(a.numpy(), axis=0))
print(np.cov(a.numpy()))



### Preprocessing

In [ ]:
device = torch.device("cuda")
device

In [ ]:
p_experiment = "/mnt/data/experiments/EfficientCN/mnist/effcn_mnist_MnistEffCapsNet_2022_02_03_00_32_47"
#p_experiment = "/mnt/data/experiments/EfficientCN/mnist/effcn_mnist_MnistCNN_R_2022_02_04_01_02_39"
p_experiment = Path(p_experiment)
p_ckpts = p_experiment / "ckpts"
p_data = Path("/mnt/data/datasets")
p_model = p_ckpts / "model_150.ckpt"
p_model.exists()

In [ ]:
model = EffCapsNet()
#model = MnistCNN_R()
model.load_state_dict(torch.load(p_model))
model = model.to(device)
model.eval()

### Kullbach-Leiber Divergence and PCA

In [ ]:
train = True
ds_train = datasets.MNIST(root=p_data, train=train, download=True, transform=T.ToTensor())

dl_train = torch.utils.data.DataLoader(ds_train, 
                                        #batch_size=len(ds_train), 
                                        batch_size=8, 
                                        shuffle=False)

In [ ]:
x,y = next(iter(dl_train))

print(x.shape)
print(y.shape)

In [ ]:
def affine_xtrans(img, target, range=[-5.,5.,1]):
    arange = np.arange(range[0],(range[1]+range[2]),range[2])
    x_trans = torch.zeros([len(arange),img.shape[-3],img.shape[-2],img.shape[-1]])
    l_target = torch.zeros(len(arange))

    for i, l in enumerate(arange):
        x_trans[i] = T.functional.affine(img=img, angle=0, translate=[l,0], scale=1.,shear=0)
        l_target[i] = target
    
    return x_trans, l_target

def affine_ytrans(img, target, range=[-5.,5.,1]):
    arange = np.arange(range[0],(range[1]+range[2]),range[2])
    y_trans = torch.zeros([len(arange),img.shape[-3],img.shape[-2],img.shape[-1]])
    l_target = torch.zeros(len(arange))

    for i, l in enumerate(arange):
        y_trans[i] = T.functional.affine(img=img, angle=0, translate=[0,l], scale=1.,shear=0)
        l_target[i] = target
    
    return y_trans, l_target

def affine_rot(img, target, range=[-25.,25.,1]):
    arange = np.arange(range[0],(range[1]+range[2]),range[2])
    rot = torch.zeros([len(arange),img.shape[-3],img.shape[-2],img.shape[-1]])
    l_target = torch.zeros(len(arange))

    for i, l in enumerate(arange):
        rot[i] = T.functional.affine(img=img, angle=l, translate=[0,0], scale=1.,shear=0)
        l_target[i] = target
    
    return rot, l_target

def affine_scale(img, target, range=[0.75,1.25,0.05]):
    arange = np.arange(range[0],(range[1]+range[2]),range[2])
    scale = torch.zeros([len(arange),img.shape[-3],img.shape[-2],img.shape[-1]])
    l_target = torch.zeros(len(arange))

    for i, l in enumerate(arange):
        scale[i] = T.functional.affine(img=img, angle=0, translate=[0,0], scale=l,shear=0)
        l_target[i] = target
    
    return scale, l_target

def affine_shear(img, target, range=[-10.,10.,2]):
    arange = np.arange(range[0],(range[1]+range[2]),range[2])
    shear = torch.zeros([len(arange),img.shape[-3],img.shape[-2],img.shape[-1]])
    l_target = torch.zeros(len(arange))

    for i, l in enumerate(arange):
        shear[i] = T.functional.affine(img=img, angle=0, translate=[0,0], scale=1,shear=l)
        l_target[i] = target
    
    return shear, l_target    

In [ ]:
x_aff, y_aff = affine_xtrans(x[0],y[0], range=[-2.,2.,0.5])

x_aff.shape

In [ ]:
def cov_uh_trans(uh):
    """
    uh in [k,n,m]
    k -> number of transformed images
    n -> number of output classes
    m -> number of capsul values
    """

    uh_mean = uh.mean(dim=0)
    z = uh - uh_mean
    c_k = torch.einsum('...ij, ...ik -> ...jk', z,z)
    c = torch.einsum('ijk -> jk', c_k) / c_k.shape[0]

    return c

In [ ]:
###Test for model = MnistCNN_R()

affine = affine_xtrans

#generate aff transforms
x_aff, y_aff = affine(x[0],y[0])

#Generate Caps from affine Transform
x_aff = x_aff.to(device)
uh_aff, _ = model.forward(x_aff)


print(uh_aff.shape)
print(uh_aff)
print(y_aff)



#KL-Divergence
uh_aff_th = uh_aff[:,y[0]]
#Variance over each dimension
var_uh_aff = torch.var(uh_aff_th, dim=EcnDecoder0)
#Kullback-Leibler-Divergenz
kl = (var_uh_aff * torch.log((var_uh_aff/0.1)))
#kl_div.append(kl.tolist())


print(uh_aff_th)
print(uh_aff_th.shape)
print(var_uh_aff)
print(var_uh_aff.sum())
print(torch.log((var_uh_aff/1)))
print(kl)


In [ ]:
affine = affine_rot
kl_div = []

pbar = tqdm(dl_train, bar_format='{bar:10}{r_bar}{bar:-10b}')

#load batchwise
for x, y in pbar:
    #calculate staistical vals
    for i, img in enumerate(x):
        #generate aff transforms
        x_aff, y_aff = affine(x[i],y[i])
        
        #Generate Caps from affine Transform
        x_aff = x_aff.to(device)
        uh_aff, _ = model.forward(x_aff)

        #KL-Divergence
        uh_aff_th = uh_aff[:,y[i]]
        #Variance over each dimension
        var_uh_aff = torch.var(uh_aff_th, dim=0)
        #Kullback-Leibler-Divergenz
        kl = (var_uh_aff * torch.log((var_uh_aff)))
        kl_div.append(kl.tolist())


kld_mean = torch.tensor(kl_div).mean()

In [ ]:
kld_mean

In [ ]:
affine = affine_rot
pca_eig = []
kl_div = []

pbar = tqdm(dl_train, bar_format='{bar:10}{r_bar}{bar:-10b}')

#load batchwise
for x, y in pbar:
    #calculate staistical vals
    for i, img in enumerate(x):
        #generate aff transforms
        x_aff, y_aff = affine(x[i],y[i])
        
        #Generate Caps from affine Transform
        x_aff = x_aff.to(device)
        uh_aff, _ = model.forward(x_aff)

        #PCA
        #Covariance from Caps
        cov_uh = cov_uh_trans(uh_aff)
        #Eigenvals
        eig, v_eig = torch.linalg.eig(cov_uh)
        sig = eig.float() / eig.float().sum()
        #PCA eigenvalues
        pca_eig.append(sig.tolist())


        #KL-Divergence
        #Caps from valid
        uh_aff_th = uh_aff[:,y[i],:]
        #Variance over each dimension
        var_uh_aff = torch.var(uh_aff_th, dim=0)
        #Variance normalized
        nor_uh_aff = var_uh_aff / var_uh_aff.sum()
        #uniform prior
        uni_p = 1/nor_uh_aff.shape[0]
        #Kullback-Leibler-Divergenz
        kl = (nor_uh_aff * torch.log((nor_uh_aff/uni_p))).sum()
        kl_div.append(kl.tolist())


pca_mean = torch.tensor(pca_eig).mean(dim=0)
kld_mean = torch.tensor(kl_div).mean()

In [ ]:
print(len(kl_div))
print(kld_mean)
print(len(pca_eig))
print(pca_mean)

kl_div[0]

In [ ]:
stats = {
    "model": [str(p_model)],
    "dataset": [str(p_data)],
    "train": [train],
    "affine": [affine.__name__],
    "pca": {
        'pca_eig': pca_eig,
        'pca_mean': pca_mean,
    },
    "kld": {
        'kld_val': kl_div,
        'kld_mean': kld_mean,
    }
}
s =  DottedDict(stats)

In [ ]:
s =  DottedDict(stats)

In [ ]:
#pprint.pp(s)

In [ ]:
p_semcomp = Path("/mnt/data/experiments/EfficientCN/sem_comp")
p_stats = p_semcomp / 'semcomp_mnist_{tr}_{da}'.format(tr=affine.__name__, da=datetime.datetime.fromtimestamp(time.time()).strftime('%Y_%m_%d_%H_%M_%S'))

mkdir_directories([p_stats], parents=True, exist_ok=True)

In [ ]:
p_stats

In [ ]:
file1 = open(p_stats /'stats.pkl', 'wb')
pickle.dump(stats, file1)
file1.close()


In [ ]:
plt.plot(pca_mean.detach().numpy(),".")
plt.tick_params(colors="w")
plt.title(str(affine.__name__) + ", kld_mean: " + str(kld_mean.item()),color="w")

plt.savefig(p_stats /'eigenvals_mean.png')